In [1]:
import nltk
from nltk.corpus import stopwords
import string
import pandas as pd
import numpy as np
import math

# Load data
This data is downloaded from the site https://www.kaggle.com/crowdflower/twitter-user-gender-classification.

In [2]:

# data = pd.read_csv('gender-classifier-DFE-791531.csv',encoding='latin1')

data = pd.read_csv('data.csv',encoding='latin1')
data.head()
data.info()

ParserError: Error tokenizing data. C error: Expected 31 fields in line 228, saw 40


In [ ]:
data.gender = data.gender.replace('male','1')
data.gender = data.gender.replace('female','0')
data.gender = data.gender.replace('brand','nan')
all_descriptions = data['description']
all_tweets = data['text']
all_genders = data['gender']
all_gender_confidence = data['discrete-gender:confidence']
valid_tweet_description_gender = []

# Cleaning Data

### Creation of bag of words

In [ ]:
# Creation of bag of words for the description
bag_of_words = []
i = 0  # for the index of the row
stop = stopwords.words('english')
for tweet in all_tweets:
    description = all_descriptions[i]
    gender = all_genders[i]
    gender_confidence = all_gender_confidence[i]
    
    # remove the rows which has an empty tweet and description
    # remove the rows with unknown or empty gender
    if (str(tweet) == 'nan' and str(description) == 'nan') or str(gender) == 'nan' or str(gender) == 'unknown':
        i=i+1
        continue
    
    if str(tweet) == 'nan':
        tweet = ''
    if str(description) == 'nan':
        description = ''
    
    # removal of punctuations
    for punctuation in string.punctuation:
        if punctuation in tweet:
            tweet = tweet.replace(punctuation, " ")
        if punctuation in description:
            description = description.replace(punctuation, " ")
            
    # adding the word to the bag except stopwords
    for word in tweet.split():
        if word.isalpha() and word.lower() not in stop:
            bag_of_words.append(word.lower())
    for word in description.split():
        if word.isalpha() and word.lower() not in stop:
            bag_of_words.append(word.lower())
    
    # using tweet and description for classification
    valid_tweet_description_gender.append((tweet+" "+description , gender))
    i=i+1

print(len(bag_of_words))
print(len(valid_tweet_description_gender))

valid_tweet_description_gender

### get top 100 words which will act as our features of each sentence

In [ ]:
# get top 100 words which will act as our features of each sentence
bag_of_words = nltk.FreqDist(bag_of_words)
top_words = []
for word in bag_of_words.most_common(100):
    if word[0] != 'co' and word[0] != 'https':
        top_words.append(word[0])

top_words[:10]

### Finding our features within each datapoint

In [ ]:
def find_features(top_words, text):
    feature = {}
    for word in top_words:
        feature[word] = word in text.lower()
        
    return feature

### Frequency of our features in each datapoint

In [ ]:
import re
def find_freq(top_words, text):
    feature = {}
    frequency= {}
    for word in top_words:
        a = text.lower()
        frequency[word] = a.count(word) # 1
        
    return frequency

In [ ]:
freq_set = [(find_freq(top_words, text), gender) for (text, gender) in valid_tweet_description_gender]

### Creation of Sparse Matrix 

In [ ]:
matrix=[]
for i in range(len(freq_set)):
    matrix.append(list(freq_set[i][0].values()))
    
matrix = np.matrix(matrix)
matrix

v = []
for i in range(len(freq_set)):
    v.append(int(freq_set[i][1]))
    
# matrix = np.matrix(matrix)
v = np.array(v)

### Training and Testind data 

In [ ]:
trainingset =matrix[:int(len(freq_set)*3/5)]
validationset=matrix[int(len(freq_set)*3/5):int(len(freq_set)*4/5)]
testingset = matrix[int(len(freq_set)*4/5):]

In [ ]:
Dataframe=pd.DataFrame(trainingset) 
Dataframe1=pd.DataFrame(testingset)
Dataframe2=pd.DataFrame(validationset)


In [ ]:
x_train=Dataframe
x_train=(x_train-np.min(x_train,axis=0))/(np.max(x_train,axis=0)-np.min(x_train,axis=0))

y_train=v[:int(len(freq_set)*3/5)]
y_train=y_train[:,np.newaxis]
# Y=Y-1

x_validation=Dataframe2
x_validation=(x_validation-np.min(x_validation,axis=0))/(np.max(x_validation,axis=0)-np.min(x_validation,axis=0))

y_validation=v[int(len(freq_set)*3/5):int(len(freq_set)*4/5)]
y_validation=y_validation[:,np.newaxis]

x_test = Dataframe1
x_test=(x_test-np.min(x_test,axis=0))/(np.max(x_test,axis=0)-np.min(x_test,axis=0))

y_test =v[int(len(matrix)*4/5):]
y_test=y_test[:,np.newaxis]

# Neural Network

In [3]:
shape_col = x_train.shape[1]                       #this is the number of features

def init_weight(shape_col):
    np.random.seed(7)                             #this is for the purpose of keepin the random numbers contant
    weights = np.array (np.random.rand(shape_col,1))
    bias = np.array( np.random.rand(1) )
    init_list = [weights,bias]
    return init_list
      
def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x)*(1-sigmoid(x))

def Neuaral_network( network_parameters, weights_data, target):
    #target = target[:, np.newaxis]
    lr = 0.005
    for epoch in range(1000):
        value = np.dot(network_parameters ,weights_data[0]) + weights_data[1]
        acti_value = sigmoid(value)
        deriv = sigmoid_derivative(acti_value)
        error = acti_value - target
        #print(error.sum())
        z_del = error * deriv
        weights_data[0] = weights_data[0] - lr*np.dot(network_parameters.T, z_del) 

        for num in z_del:
            weights_data[1] = weights_data[1] - lr*num
        
    return np.array(weights_data)
    
trained_weight = Neuaral_network( x_train , init_weight(shape_col), y_train )
trained_weightval = Neuaral_network( x_validation , trained_weight, y_validation )



def predict_NN(Neuaral_network,test_data):
    predicting = []
    result = sigmoid(np.dot(test_data, Neuaral_network[0]) + Neuaral_network[1])
    
    for i in range(len(result)):
        if(result[i]>0.5):
            predicting.append(1)
        else:
            predicting.append(0)
            
    return np.array(predicting)
    
def accurracy_NN(predi,y):
    p=predi[:, np.newaxis]
    return np.count_nonzero(p == y)/len(y)

def hiddenLayer(real, pred, realV, predV):
    
    count = 0
    for i in range(len(real)):
        if(real[i] == realV and pred[i] == predV):
            count += 1;
    return count;

def confusionMatrix(realTarget, predictedTarget):
    realTarget = np.array(realTarget)
    predictedTarget = np.array(predictedTarget)
    
    
    values = []
    matrix = []
    uniqueValues = np.array(np.unique(realTarget))
    for i in range(len(uniqueValues)):
        for j in range(len(uniqueValues)):
            values.append(hiddenLayer(realTarget, predictedTarget, uniqueValues[i], uniqueValues[j]))
        matrix.append(values)
        values = []
    
    m = pd.DataFrame({'pred female, male': matrix}, index =['Actual female', 'Actual male'])
    return m

print("Neural Network classification accuracy on Training data:",accurracy_NN( predict_NN(trained_weight , x_train), y_train)*100)
print("Neural Network classification accuracy on Validation data:",accurracy_NN( predict_NN(trained_weightval , x_validation), y_validation)*100)
print("Neural Network classification accuracy on Testing data:",accurracy_NN( predict_NN(trained_weightval, x_test), y_test)*100)
#print("Neural network Accuracy:", accurracy_NN( predict_NN(trained_weight , x_test), y_test) )
print("Confusion Matrix:\n", 
        confusionMatrix(y_test,predict_NN(trained_weightval, x_test)))

NameError: name 'x_train' is not defined

# Logistic Regression

In [4]:
#sigmoid function
# X = np.c_[X, Y]
def sigmoid(x,theta):
#     theta = theta[:,np.newaxis]
    f=(1/(1+np.exp(-np.dot(x,theta))))
    return f[:,np.newaxis]

#gradient descent
def gd(theta,alpha=0.1,iterations=500):
    N=len(x_train)
    h=sigmoid
    i=0
    while True:
        error=(h(x_train,theta)-y_train)*x_train
        grad=np.mean(x_train,axis=0)
        theta_prior=theta.copy()
        theta=theta-alpha*grad
        res=theta-theta_prior
        #print(theta)
        if i>100:
            return theta
        i=i+1


def gdv(theta,alpha=0.1,iterations=500):
    N=len(x_validation)
    h=sigmoid
    i=0
    while True:
        error=(h(x_validation,theta)-y_validation)*x_validation
        grad=np.mean(x_train,axis=0)
        theta_prior=theta.copy()
        theta=theta-alpha*grad
        res=theta-theta_prior
        #print(theta)
        if i>100:
            return theta
        i=i+1
        
def predict(x,theta):
    h=sigmoid(x,theta)
    h[h>0.5]=1
    h[h<=0.5]=0
    return h

def accurracy(x,theta,y):
    p=predict(x,theta)
    return np.count_nonzero(p==y)/len(x)

def hiddenLayer(real, pred, realV, predV):
    
    count = 0
    for i in range(len(real)):
        if(real[i] == realV and pred[i] == predV):
            count += 1;
    return count;

def confusionMatrix(realTarget, predictedTarget):
    realTarget = np.array(realTarget)
    predictedTarget = np.array(predictedTarget)
    
    
    values = []
    matrix = []
    uniqueValues = np.array(np.unique(realTarget))
    for i in range(len(uniqueValues)):
        for j in range(len(uniqueValues)):
            values.append(hiddenLayer(realTarget, predictedTarget, uniqueValues[i], uniqueValues[j]))
        matrix.append(values)
        values = []
    
    m = pd.DataFrame({'pred female, male': matrix}, index =['Actual female', 'Actual male'])
    return m
        
theta=np.random.randn(len(x_train.T[0]),)
theta=gd(theta,alpha=0.2)
print("Logistic Regression classification accuracy on Training data", accurracy(x_train,theta,y_train)*100)
theta=gdv(theta,alpha=0.2)
print("Logistic Regression classification accuracy on Validation data", accurracy(x_validation,theta,y_validation)*100)

print("Logistic Regression classification accuracy on Testing data", accurracy(x_test,theta,y_test)*100)
print("Confusion Matrix:\n", 
        confusionMatrix(y_test,predict(x_test,theta)))

NameError: name 'x_train' is not defined

# Naive Bayes

In [ ]:
class NaiveBayes():

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)

        # calculate mean, var, and prior for each class
        self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self._var = np.zeros((n_classes, n_features), dtype=np.float64)
        self._priors =  np.zeros(n_classes, dtype=np.float64)

        for idx, c in enumerate(self._classes):
            X_c = X[y==c]
            self._mean[idx, :] = X_c.mean(axis=0)
            self._var[idx, :] = X_c.var(axis=0)
            self._priors[idx] = X_c.shape[0] / float(n_samples)

    def predict(self, x):
        y_pred = [self._predict(a) for a in x]
        
        return np.array(y_pred)

    def _predict(self, x):
        posteriors = []

        # calculate posterior probability for each class
        for idx, c in enumerate(self._classes):
            prior = np.log(self._priors[idx])
            posterior = np.sum(np.log(self._pdf(idx, x)))
            posterior = prior + posterior
            posteriors.append(posterior)
            
        # return class with highest posterior probability
        return self._classes[np.argmax(posteriors)]
            

    def _pdf(self, class_idx, x):
        mean = self._mean[class_idx]
        var = self._var[class_idx]
        numerator = np.exp(- (x-mean)**2 / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator

In [ ]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

nb = NaiveBayes()

nb.fit(x_train, y_train)
prediction = nb.predict(x_train)
print("Naive Bayes classification accuracy on Training data", accuracy(y_train, prediction))

nb.fit(x_validation, y_validation)
prediction = nb.predict(x_validation)
print("Naive Bayes classification accuracyon Validation data", accuracy(y_validation, prediction))

prediction = nb.predict(x_test)
print("Naive Bayes classification accuracy on Testing data", accuracy(y_test, prediction))


# Decision Tree Algorithm

In [ ]:
#removing columns with many unique values because the slow down computation
df = pd.read_csv('data.csv',encoding='latin1')
df.drop(labels=['gender:confidence'], axis=1, inplace = True)
df.drop(labels=['fav_number'], axis=1, inplace = True)
df.drop(labels=['profile_yn:confidence'], axis=1, inplace = True)
df.drop(labels=['tweet_count'], axis=1, inplace = True)
df.drop(labels=['Unnamed: 0'], axis=1, inplace = True)
df.drop(labels=['_unit_id'], axis=1, inplace = True)
df.drop(labels=['_last_judgment_at'], axis=1, inplace = True)
df.drop(labels=['created'], axis=1, inplace = True)
df.drop(labels=['description'], axis=1, inplace = True)
df.drop(labels=['link_color'], axis=1, inplace = True)
df.drop(labels=['name'], axis=1, inplace = True)
df.drop(labels=['profileimage'], axis=1, inplace = True)
df.drop(labels=['sidebar_color'], axis=1, inplace = True)
df.drop(labels=['text'], axis=1, inplace = True)
df.drop(labels=['tweet_coord'], axis=1, inplace = True)
df.drop(labels=['tweet_created'], axis=1, inplace = True)
df.drop(labels=['tweet_location'], axis=1, inplace = True)
df.drop(labels=['user_timezone'], axis=1, inplace = True)
df.drop(labels=['tweet_id'], axis=1, inplace = True)

#move target column to the end
df["gender"].value_counts().sum()
gender_column = df['gender']
df.drop(labels=['gender'], axis=1, inplace = True)
df.insert(11, 'gender', gender_column)

#make retweet_count column to be discrete
bins4 = [-math.inf,0, 33, 66, 99, 132, 165, 198, 231, 264, 297, 330, math.inf]
df['retweet_count'] = pd.cut(df['retweet_count'], bins4)
df['retweet_count'] = df['retweet_count'].astype(str)

# remove brand
df = df.drop(df[(df['gender'] == 'brand')].index)

In [ ]:
def split_train_test_df(df, train_size_ratio):
    train_size = int(df.shape[0]*train_size_ratio)
    df_train = df.head(train_size)

    df_test_vidation = df.tail(df.shape[0]-train_size)

    validation_size = int(0.2*df.shape[0])
    df_validation = df.head(validation_size)

    test_size = df.shape[0] -train_size-validation_size
    df_test = df.tail(test_size)
    
    return df_train,df_validation,df_test

def split(df,featureName, featureValue):
    df_right = df[(df[featureName] == featureValue)]
    df_left = df[(df[featureName] != featureValue)]
    return df_right,df_left 

def main_Entropy(target):
    targetArray = target['gender'].value_counts().values
    total = targetArray.sum()
    s = 0
    if(total != 0):
        for i in targetArray:
            p1 = i/total
            if (p1 == 0):
                p2 = 0
            else :
                p2 = math.log2(p1)
            s = s + p1*p2

        s = -1*s
    return s

def subset_Entropy(df,FeatureName, subsetValue):
    totalSubsetValue = df[(df[FeatureName] == subsetValue)].shape[0]
    
    if(totalSubsetValue != 0):
        #Male probability
        MP1 = df[(df[FeatureName] == subsetValue) & (df['gender'] == 'male')].shape[0]/totalSubsetValue

        if (MP1 == 0):
            MP2 = 0
        else :
            MP2 = math.log2(MP1)

        #Female probability
        MF1 = df[(df[FeatureName] == subsetValue) & (df['gender'] == 'female')].shape[0]/totalSubsetValue
        if (MF1 == 0):
            MF2 = 0
        else :
            MF2 = math.log2(MF1)

        s = -1*(MP1*MP2 + MF1*MF2)
    else:
        s = 0
    
    return s

def gain(df,FeatureName):
    mainEntropy = main_Entropy(df)
    
    total = df.shape[0]
    if(total != 0):
        featureArray = df[[FeatureName]].values.flatten()

        uniqueSubsetValues = np.unique(featureArray)

        sss = 0

        for i in uniqueSubsetValues:
            ddd = np.count_nonzero(featureArray == i)
            subsetEntropy = subset_Entropy(df, FeatureName, i)
            sss = sss + ddd*subsetEntropy

        gain =  mainEntropy - (1/total)*sss
    else:
        gain = 0
    return gain

def get_split_feature_value(df):
    maxGain = -1000
    features = df.columns
    feature_size = df.shape[1]
    for i in range(feature_size-1):
        uniqueSubsetValues = np.unique(df[[features[i]]].values.flatten())
        for value in uniqueSubsetValues:
            df_right, df_left = split(df, features[i], value)
            left_gain = gain(df_right,features[i])
            right_gain = gain(df_left,features[i])
            total_gain = left_gain+right_gain

            if total_gain > maxGain:
                maxGain = total_gain
                split_feature = features[i]
                split_value = value
    return split_feature,split_value

def decision_tree_algorithm(df, level=0, depth=4):
    
    uniqueSubsetValues = len(np.unique(df[['gender']].values.flatten()))
    if (level == depth or uniqueSubsetValues == 1 or (df.shape[0] < 10)):
        targets, counts_targets = np.unique(df[['gender']].values.flatten(), return_counts = True)
        return targets[counts_targets.argmax()]
    else:
        level = level+1
        
        sf, sv = get_split_feature_value(df)
        
        df_right, df_left = split(df, sf, sv)
        
        question = "{} == {}".format(sf, sv)
        
        # instantiate sub-tree
        sub_tree = {question: []}
        
        if(df_right.shape[0] != 0):
            yes_answer = decision_tree_algorithm(df_right, level, depth)
            
        if(df_left.shape[0] != 0):
            no_answer = decision_tree_algorithm(df_left, level, depth)
            
        if(df_right.shape[0] != 0 and df_left.shape[0] != 0):
            if yes_answer == no_answer:
                sub_tree = yes_answer
            else:
                sub_tree[question].append(yes_answer)
                sub_tree[question].append(no_answer)
                
        elif(df_right.shape[0] != 0 and df_left.shape[0] == 0):
            sub_tree[question].append(yes_answer)
            
        elif(df_right.shape[0] == 0 and df_left.shape[0] != 0):
            sub_tree[question].append(no_answer)
    
        else:
            sub_tree[question].append("Error")
           
        return sub_tree
    
    
def predict(data_point,treee):
    question1 = list(treee.keys())[0].split(" ")
    question = list(treee.keys())[0]
    feature_name = question1[0]
    value = question1[2]
    
    if data_point[feature_name] == value:
        answer = treee[question][1]
    else:
        answer = treee[question][0]

    if not isinstance(answer, dict):
        return answer
    else:
        residual_tree = answer
        return predict(data_point,residual_tree)
    
    
def accuracy_calculate(df, treee):
    df["predicted"] = df.apply(predict, args=(treee,), axis=1)
    count = 0
                        
    confusion_matrix = [[0,1],
                        [0,1]]
    for ii in range(df.shape[0]):
        if(df['gender'].values[ii] == df['predicted'].values[ii]):
            count = count + 1
            
    accuracy = (count/df.shape[0])*100
    
    return accuracy, df['predicted'].values


def hiddenLayer(real, pred, realV, predV):
    
    count = 0
    for i in range(len(real)):
        if(real[i] == realV and pred[i] == predV):
            count += 1;
    return count;

def confusionMatrix(realTarget, predictedTarget):
    realTarget = np.array(realTarget)
    predictedTarget = np.array(predictedTarget)
    
    
    values = []
    matrix = []
    uniqueValues = np.array(np.unique(realTarget))
    for i in range(len(uniqueValues)):
        for j in range(len(uniqueValues)):
            values.append(hiddenLayer(realTarget, predictedTarget, uniqueValues[i], uniqueValues[j]))
        matrix.append(values)
        values = []
    m = pd.DataFrame({'pred female, male': matrix}, index =['Actual female', 'Actual male'])
    return m


df_train,df_validation, df_test = split_train_test_df(df, 0.6)
tree = decision_tree_algorithm(df_train)
acc_train, pred_train = accuracy_calculate(df_train, tree)
acc_validation, pred_validation = accuracy_calculate(df_validation, tree)
acc_test, pred_test = accuracy_calculate(df_test, tree)

print("Decision Tree Algorithm classification accuracy on Training data", acc_train)

print("Decision Tree Algorithm classification accuracy on Validation data", acc_validation)

print("Decision Tree Algorithm classification accuracy on Testing data", acc_test)
print("Confusion Matrix:")
confusionMatrix(df_test['gender'].values, pred_test)
